<a href="https://colab.research.google.com/github/pstansfeld/MemProtMD/blob/main/MemProtMD_insane_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Upload a PDB coordinate file
import os
import sys

!python3 -m pip install py3dmol

import py3Dmol
from google.colab import files
sys.path.append('/usr/local/lib/python3.7/site-packages/')

os.chdir('/content/')
upload = files.upload()
#@title Predict Membrane Orientation
mview = py3Dmol.view(800, 400)  
filename = next(iter(upload))
name = os.path.splitext(filename)[0]
working_dir = '/content/' + name + '/'
os.makedirs(working_dir, exist_ok=True)
os.rename(filename, working_dir + filename)
os.chdir(working_dir)
mol1 = open(working_dir + filename, 'r').read()
mview.addModel(mol1,'pdb')
mview.setStyle({'cartoon': {'color':'spectrum'}})
mview.setStyle({'resn':'DUM'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

In [2]:
#@title Install dependencies
%%capture
os.chdir('/content/')
if not os.path.isdir("/content/memembed/"):
  !apt-get update -y
  !apt-get install dssp
  !git clone https://github.com/timnugent/memembed
  %cd memembed
  !make
  %cd ../
  !python3 -m pip install pdb2pqr
  !python3 -m pip install vermouth
  !python3 -m pip install GromacsWrapper
  !python3 -m pip install MDAnalysis 
  !git clone https://github.com/pstansfeld/cg2at
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/martini_v300.zip
  !unzip -o martini_v300.zip
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/insane3.py
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/gromacs.zip
  !unzip -o gromacs.zip
  !ln -s /content/content/gromacs/bin/gmx /usr/bin/gmx

import gromacs
import MDAnalysis
import glob
import shutil

In [3]:
#@title Predict Membrane Orientation
os.chdir(working_dir)
os.system('/content/memembed/bin/memembed -o memembed.pdb ' + filename) 
mview = py3Dmol.view(800, 400)  
mol1 = open(working_dir + '/memembed.pdb', 'r').read()
mview.addModel(mol1,'pdb')
mview.setStyle({'cartoon': {'color':'spectrum'}})
mview.setStyle({'resn':'DUM'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
#@title Set-up Coarse-Grained Membrane Protein System
%%capture
os.chdir(working_dir)

for file in glob.glob(r'/content/martini*.itp'):
    print(file)
    shutil.copy(file, working_dir)

gromacs.make_ndx(f='memembed.pdb', o='index.ndx', input=('del 0', 'del 1-100','rDUM','q'),backup=False)
gromacs.editconf(f='memembed.pdb',o='centered.pdb',n='index.ndx',c=True,d=3,input=(0,0),backup=False)

u = MDAnalysis.Universe('centered.pdb')
x = round(u.dimensions[0]/10)
y = round(u.dimensions[1]/10)
z = round(u.dimensions[2]/10)

gromacs.confrms(f2='memembed.pdb', f1='centered.pdb', one=True, o='aligned.gro', input=(3,3),backup=False)
gromacs.editconf(f='aligned.gro', o='protein.pdb',label='A',resnr=1,n=True,input=(0,0),backup=False)

v = MDAnalysis.Universe('aligned.gro')
dum = v.select_atoms('resname DUM')

dm = (z/2) - (round(dum.center_of_mass()[2]/10))

with open('protein.pdb', 'r') as file :
  filedata = file.read()
filedata = filedata.replace('HSE', 'HIS')
filedata = filedata.replace('HSD', 'HIS')
filedata = filedata.replace('MSE', 'MET')
filedata = filedata.replace(' SE ', ' SD ')
with open('protein.pdb', 'w') as file:
  file.write(filedata)

!martinize2 -f protein.pdb -dssp mkdssp -ff martini3001 -x protein-cg.pdb -o protein-cg.top -elastic -ef 500 -eu 1.0 -el 0.5 -ea 0 -ep 0 -merge A -maxwarn 100000

!sed -e 's/^molecule.*/Protein 1/g' molecule*.itp >  protein-cg.itp

os.system('python2 /content/insane3.py -l POPE:7 -l POPG:2 -l CARD:1 -salt 0.15 -sol W -o CG-system.gro -p topol.top -f protein-cg.pdb -center -x %s -y %s -z %s -dm %s' % (x, y, z, dm))

replacements = {'NA+':'NA', 'CL-':'CL', '#include "martini_v3.itp"':'#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n'}
lines = []
with open('topol.top') as infile:
    for line in infile:
        for src, target in replacements.items():
            line = line.replace(src, target)
        lines.append(line)
with open('topol.top', 'w') as outfile:
    for line in lines:
        outfile.write(line)
with open('em.mdp','w') as em:
            em.write('integrator = steep\nnsteps = 5000\nemtol = 1000\nemstep = 0.001')

gromacs.grompp(f='em.mdp',o='em.tpr',c='CG-system.gro',maxwarn='-1',backup=False,v=True)

gromacs.mdrun(deffnm='em', c='CG-system.pdb',backup=False)

gromacs.trjconv(f='CG-system.pdb', o='CG-system.pdb', pbc='res', s='em.tpr', conect=True, input='0',backup=False)

%mkdir MD

gromacs.make_ndx(f='CG-system.pdb', o='index.ndx', input=('del 0', 'del 1-40', '0|rPOP*','1&!0','!1','del 1','name 1 Lipid','name 2 SOL_ION','q'),backup=False)

with open('cgmd.mdp','w') as md:
            md.write('integrator = md\ntinit = 0.0\ndt = 0.02\nnsteps = 10000\nnstxout = 0\nnstvout = 0\nnstfout = 0\nnstlog = 50000\nnstenergy = 50000\nnstxout-compressed = 50000\ncompressed-x-precision = 10000\nnstlist  = 10\nns_type  = grid\npbc   = xyz\ncoulombtype  = Reaction_field\nrcoulomb_switch = 0.0\nrcoulomb  = 1.1\nepsilon_r  = 15\nvdw_type  = cutoff\nrvdw_switch  = 0.9\nrvdw   = 1.1\ncutoff-scheme = verlet\ncoulomb-modifier = Potential-shift\nvdw-modifier  = Potential-shift\nepsilon_rf  = 0\nverlet-buffer-tolerance = 0.005\ntcoupl  = v-rescale\ntc-grps  = PROTEIN LIPID SOL_ION\ntau_t  = 1.0 1.0 1.0\nref_t  = 310 310 310\nPcoupl  = c-rescale\nPcoupltype  = semiisotropic\ntau_p  = 12.0\ncompressibility = 3e-4 3e-4\nref_p  = 1.0 1.0\ngen_vel  = yes\ngen_temp  = 310\ngen_seed  = -1\nconstraints  = none\nconstraint_algorithm = Lincs\ncontinuation  = no\nlincs_order  = 4\nlincs_warnangle = 30\n')

gromacs.grompp(f='cgmd.mdp',o='MD/md',c='CG-system.pdb',maxwarn=-1, n='index.ndx',backup=False)  

In [5]:
#@title Equilibrate Membrane Protein System
os.chdir(working_dir + 'MD')
gromacs.mdrun(deffnm='md',backup=False, v=True, nsteps=10000)
mview = py3Dmol.view(800, 400)  
mol1 = open(working_dir + 'MD/md.gro', 'r').read()
mview.addModel(mol1,'gro')
mview.setStyle({'cartoon': {'color':'spectrum'}})
mview.setStyle({'atom':'PO4'},{'sphere':{}})
mview.setStyle({'atom':'BB'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

for file in glob.glob(r'#*'):
    os.remove(file)
    
os.chdir(working_dir)
for file in glob.glob(r'#*'):
    os.remove(file)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title Convert back to Atomistic
os.chdir(working_dir + 'MD/')
shutil.rmtree('CG2AT', ignore_errors=True)
os.system('/content/cg2at/cg2at -o align -w tip3p -c md.gro -a ../protein.pdb -loc CG2AT -ff charmm36-jul2020-updated -fg martini_3-0_charmm36')
os.rename(working_dir + 'MD/md.gro', working_dir + name + '-cgmd.gro')
os.rename(working_dir + 'MD/md.tpr', working_dir + name + '-cgmd.tpr')
os.rename(working_dir + 'memembed.pdb', working_dir + name + '-memembed.pdb')
os.chdir(working_dir)
shutil.rmtree('Atomistic', ignore_errors=True)
shutil.copytree(working_dir + 'MD/CG2AT/FINAL/', working_dir + 'Atomistic/')
os.chdir(working_dir + 'Atomistic/')
os.rename(working_dir + 'Atomistic/final_cg2at_aligned.pdb',working_dir + 'Atomistic/' + name + '-System.pdb')
os.rename(working_dir + 'Atomistic/topol_final.top', working_dir + 'Atomistic/topol.top')
gromacs.make_ndx(f=working_dir + 'Atomistic/' + name + '-System.pdb', o='index.ndx', input=('del 2-40', 'rSOL|rNA*|rCL*','1|2','0&!3','del 3','name 2 water_and_ions','name 3 Lipid','q'),backup=False)
with open(working_dir + 'Atomistic/topol.top', 'r') as file :
  filedata = file.read()
filedata = filedata.replace('../FINAL/', '')
with open(working_dir + 'Atomistic/topol.top', 'w') as file:
  file.write(filedata)
!wget https://raw.githubusercontent.com/pstansfeld/MemProtMD/main/mdp_files/500ns.mdp
!wget https://raw.githubusercontent.com/pstansfeld/MemProtMD/main/mdp_files/10ns-pr.mdp
mview = py3Dmol.view(800, 400)  
mol1 = open(working_dir + 'Atomistic/' + name + '-System.pdb', 'r').read()
mview.addModel(mol1,'pdb')
mview.setStyle({'cartoon': {'color':'spectrum'}})
mview.setStyle({'atom':'P'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

In [ ]:
#@title Download Zip
os.chdir(working_dir)
shutil.rmtree('MD', ignore_errors=True)
os.remove('em.trr')
os.remove('em.edr')
os.remove('em.tpr')
os.remove('em.log')
os.remove('centered.pdb')
os.remove('protein.pdb')
os.remove('molecule_0.itp')
os.remove('mdout.mdp')
os.remove('aligned.gro')
os.remove('protein-cg.top')
os.system('zip -r /content/' + name + '.zip ' + working_dir )
files.download('/content/' + name + '.zip')